In [ ]:
# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
!PIP_DISABLE_PIP_VERSION_CHECK=1 pip3 install kfp --upgrade --user --quiet

In [ ]:
FIRSTNAME_LASTNAME = 'firstname_lastname'
EXPERIMENT_NAME = 'Hello world! - container 1 ' + FIRSTNAME_LASTNAME

In [ ]:
# Install Kubeflow Pipelines SDK
!PIP_DISABLE_PIP_VERSION_CHECK=1 pip3 install 'kfp>=0.1.32.2' --quiet
!PIP_DISABLE_PIP_VERSION_CHECK=1 pip3 install 'pandas' --quiet --user

In [ ]:
import kfp
import kfp.components as comp
# Initializing the client
client = kfp.Client()

In [ ]:
def func_that_uses_uninstalled_package():
    '''Dummy function that imports pandas'''
    import pandas
    print('import successful')

In [ ]:
func_op = comp.func_to_container_op(func_that_uses_uninstalled_package)

#### Define the pipeline
Pipeline function has to be decorated with the `@dsl.pipeline` decorator

In [ ]:
import kfp.dsl as dsl
@dsl.pipeline(
   name='Pipeline with import errors',
   description='A toy pipeline that demonstrates import errors'
)
def pipeline(
):
    #building a one step pipeline to Visualize the results 
    dummy_task = func_op()
    

#### Compile the pipeline

In [ ]:
pipeline_func = pipeline
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'
import kfp.compiler as compiler
compiler.Compiler().compile(pipeline_func, pipeline_filename)

#### Submit the pipeline for execution

In [ ]:
#Get or create an experiment and submit a pipeline run
import kfp
client = kfp.Client()
experiment = client.create_experiment(EXPERIMENT_NAME)

In [ ]:
#Specify pipeline argument values
arguments = {}

#Submit a pipeline run
run_name = pipeline_func.__name__ + ' ' + FIRSTNAME_LASTNAME + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)

verify that the pipeline fails because pandas is not installed on the base image

#### Create the new base image

In [ ]:
with open('requirements.txt', 'w') as file:
    file.write('pandas')

In [ ]:
from kfp.containers import build_image_from_working_dir
from kfp.components import func_to_container_op

In [ ]:
# Building and pushing new container image
image_with_packages = build_image_from_working_dir(
    image_name='gcr.io/meetup-kfp/{}/image_pandas:latest'.format(FIRSTNAME_LASTNAME)
)
    
print(image_with_packages)

In [ ]:
func_op = comp.func_to_container_op(func_that_uses_uninstalled_package, base_image=image_with_packages)

In [ ]:
# You can also set up the image builder as default image so that it is always used by default
kfp.components.default_base_image_or_builder = build_image_from_working_dir

#### Define the pipeline
Pipeline function has to be decorated with the `@dsl.pipeline` decorator

In [ ]:
import kfp.dsl as dsl
@dsl.pipeline(
   name='Pipeline with import errors',
   description='A toy pipeline that demonstrates import errors'
)
def pipeline_fixed(
):
    #building a one step pipeline to Visualize the results 
    dummy_task = func_op()
    

#### Compile the pipeline

In [ ]:
pipeline_func = pipeline_fixed
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'
import kfp.compiler as compiler
compiler.Compiler().compile(pipeline_func, pipeline_filename)

#### Submit the pipeline for execution

In [ ]:
#Get or create an experiment and submit a pipeline run
import kfp
client = kfp.Client()
experiment = client.create_experiment(EXPERIMENT_NAME)

In [ ]:
#Specify pipeline argument values
arguments = {}

#Submit a pipeline run
run_name = pipeline_func.__name__ + ' ' + FIRSTNAME_LASTNAME + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)